# TCGA Data Exploration
---

Exploring the preprocessed TCGA dataset from the Pancancer paper (https://www.ncbi.nlm.nih.gov/pubmed/29625048).

The Cancer Genome Atlas (TCGA), a landmark cancer genomics program, molecularly characterized over 20,000 primary cancer and matched normal samples spanning 33 cancer types. This joint effort between the National Cancer Institute and the National Human Genome Research Institute began in 2006, bringing together researchers from diverse disciplines and multiple institutions.

## Importing the necessary packages

In [1]:
import modin.pandas as pd                  # Optimized distributed version of Pandas
import re                                  # re to do regex searches in string data
import plotly                              # Plotly for interactive and pretty plots
import plotly.graph_objs as go
from datetime import datetime              # datetime to use proper date and time formats
import os                                  # os handles directory/workspace changes
import numpy as np                         # NumPy to handle numeric and NaN operations
from tqdm import tqdm_notebook             # tqdm allows to track code execution progress
import numbers                             # numbers allows to check if data is numeric
import torch                               # PyTorch to create and apply deep learning models
from torch.utils.data.sampler import SubsetRandomSampler

Allow pandas to show more columns:

In [2]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

Set the random seed for reproducibility:

In [3]:
np.random.seed(42)

In [4]:
import os
import pandas as pd

# Set base data path
data_path = '/Users/elenalickel/Desktop/Thesis/IRR_Code/TCGA-Pancancer/'

# File names
rna_file = 'EBPlusPlusAdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.tsv'
abs_file = 'TCGA_mastercalls.abs_segtabs.fixed.txt'
cdr_file = 'TCGA-CDR-SupplementalTableS1.xlsx'

# Full paths
rna_path = os.path.join(data_path, rna_file)
abs_path = os.path.join(data_path, abs_file)
cdr_path = os.path.join(data_path, cdr_file)

# Load RNA and transpose
rna_df = pd.read_csv(rna_path, sep='\t')
rna_df = rna_df.set_index('gene_id').transpose()
print("RNA shape:", rna_df.shape)
display(rna_df.head())

# Load ABSOLUTE CNV data
abs_df = pd.read_csv(abs_path, sep='\t')
print("ABSOLUTE shape:", abs_df.shape)
display(abs_df.head())

# Load Clinical Data Resource (CDR)
cdr_df = pd.read_excel(cdr_path)
print("CDR shape:", cdr_df.shape)
display(cdr_df.head())

RNA shape: (11069, 20531)


gene_id,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,?|317712,?|340602,?|388795,?|390284,?|391343,?|391714,?|404770,?|441362,?|442388,?|553137,?|57714,?|645851,?|652919,?|653553,?|728045,?|728603,?|728788,?|729884,?|8225,?|90288,A1BG|1,A1CF|29974,A2BP1|54715,A2LD1|87769,A2M|2,A2ML1|144568,A4GALT|53947,A4GNT|51146,AAA1|404744,AAAS|8086,AACS|65985,AACSL|729522,AADAC|13,AADACL2|344752,AADACL3|126767,AADACL4|343066,AADAT|51166,AAGAB|79719,AAK1|22848,AAMP|14,AANAT|15,AARS|16,AARS2|57505,AARSD1|80755,AASDH|132949,AASDHPPT|60496,AASS|10157,AATF|26574,AATK|9625,ABAT|18,ABCA10|10349,ABCA1|19,ABCA11P|79963,ABCA12|26154,ABCA13|154664,ABCA17P|650655,ABCA2|20,ABCA3|21,ABCA4|24,ABCA5|23461,ABCA6|23460,ABCA7|10347,ABCA8|10351,ABCA9|10350,ABCB10|23456,ABCB11|8647,ABCB1|5243,ABCB4|5244,ABCB5|340273,ABCB6|10058,ABCB7|22,ABCB8|11194,ABCB9|23457,ABCC10|89845,ABCC11|85320,ABCC12|94160,ABCC13|150000,ABCC1|4363,ABCC2|1244,ABCC3|8714,ABCC4|10257,ABCC5|10057,ABCC6|368,ABCC6P1|653190,ABCC6P2|730013,ABCC8|6833,ABCC9|10060,ABCD1|215,ABCD2|225,ABCD3|5825,ABCD4|5826,ABCE1|6059,ABCF1|23,ABCF2|10061,ABCF3|55324,ABCG1|9619,ABCG2|9429,ABCG4|64137,ABCG5|64240,ABCG8|64241,ABHD10|55347,ABHD11|83451,ABHD12|26090,ABHD12B|145447,ABHD13|84945,ABHD14A|25864,ABHD14B|84836,ABHD15|116236,ABHD1|84696,ABHD2|11057,ABHD3|171586,ABHD4|63874,ABHD5|51099,ABHD6|57406,ABHD8|79575,ABI1|10006,ABI2|10152,ABI3|51225,ABI3BP|25890,ABL1|25,ABL2|27,ABLIM1|3983,ABLIM2|84448,ABLIM3|22885,ABO|28,ABP1|26,ABR|29,ABRA|137735,ABT1|29777,ABTB1|80325,ABTB2|25841,ACAA1|30,ACAA2|10449,ACACA|31,ACACB|32,ACAD10|80724,ACAD11|84129,ACAD8|27034,ACAD9|28976,ACADL|33,ACADM|34,ACADS|35,ACADSB|36,ACADVL|37,ACAN|176,ACAP1|9744,ACAP2|23527,ACAP3|116983,ACAT1|38,ACAT2|39,ACBD3|64746,ACBD4|79777,ACBD5|91452,ACBD6|84320,ACBD7|414149,ACCN1|40,ACCN2|41,ACCN3|9311,ACCN4|55515,ACCN5|51802,ACCS|84680,ACCSL|390110,ACD|65057,ACE|1636,ACE2|59272,ACER1|125981,ACER2|340485,ACER3|55331,ACHE|43,ACIN1|22985,ACLY|47,ACMSD|130013,ACN9|57001,ACO1|48,ACO2|50,ACOT11|26027,ACOT12|134526,ACOT13|55856,ACOT1|641371,ACOT2|10965,ACOT4|122970,ACOT6|641372,ACOT7|11332,ACOT8|10005,ACOT9|23597,ACOX1|51,ACOX2|8309,ACOX3|8310,ACOXL|55289,ACP1|52,ACP2|53,ACP5|54,ACP6|51205,ACPL2|92370,ACPP|55,ACPT|93650,ACR|49,ACRBP|84519,ACRC|93953,ACRV1|56,ACSBG1|23205,ACSBG2|81616,ACSF2|80221,ACSF3|197322,ACSL1|2180,ACSL3|2181,ACSL4|2182,ACSL5|51703,ACSL6|23305,ACSM1|116285,ACSM2A|123876,ACSM2B|348158,ACSM3|6296,ACSM4|341392,ACSM5|54988,ACSS1|84532,ACSS2|55902,ACSS3|79611,ACTA1|58,ACTA2|59,ACTB|60,ACTBL2|345651,ACTC1|70,ACTG1|71,ACTG2|72,ACTL6A|86,ACTL6B|51412,ACTL7A|10881,ACTL7B|10880,ACTL8|81569,ACTL9|284382,ACTN1|87,ACTN2|88,ACTN3|89,ACTN4|81,ACTR10|55860,ACTR1A|10121,ACTR1B|10120,ACTR2|10097,ACTR3|10096,ACTR3B|57180,ACTR3C|653857,ACTR5|79913,ACTR6|64431,ACTR8|93973,ACTRT1|139741,ACTRT2|140625,ACVR1|90,ACVR1B|91,ACVR1C|130399,ACVR2A|92,ACVR2B|93,ACVRL1|94,ACY1|95,ACY3|91703,ACYP1|97,ACYP2|98,ADA|100,ADAD1|132612,ADAD2|161931,ADAL|161823,ADAM10|102,ADAM11|4185,ADAM12|8038,ADAM15|8751,ADAM17|6868,ADAM18|8749,ADAM19|8728,ADAM20|8748,ADAM21|8747,ADAM21P1|145241,ADAM2|2515,ADAM22|53616,ADAM23|8745,ADAM28|10863,ADAM29|11086,ADAM30|11085,ADAM32|203102,ADAM33|80332,ADAM3A|1587,ADAM5P|255926,ADAM6|8755,ADAM7|8756,ADAM8|101,ADAM9|8754,ADAMDEC1|27299,ADAMTS10|81794,ADAMTS12|81792,ADAMTS13|11093,ADAMTS14|140766,ADAMTS15|170689,ADAMTS16|170690,ADAMTS17|170691,ADAMTS18|170692,ADAMTS19|171019,ADAMTS1|9510,ADAMTS20|80070,ADAMTS2|9509,ADAMTS3|9508,ADAMTS4|9507,ADAMTS5|11096,ADAMTS6|11174,ADAMTS7|11173,ADAMTS8|11095,ADAMTS9|56999,ADAMTSL1|92949,ADAMTSL2|9719,ADAMTSL3|57188,ADAMTSL4|54507,ADAMTSL5|339366,ADAP1|11033,ADAP2|55803,ADAR|103,ADARB1|104,ADARB2|105,ADAT1|23536,ADAT2|134637,ADAT3|113179,ADC|113451,ADCK1|57143,ADCK2|90956,ADCK4|79934,ADCK5|203054,ADCY10|55811,ADCY1|107,ADCY2|108,ADCY3|109,ADCY4|196883,ADCY5|111,ADCY6|112,ADCY7|113,ADCY8|114,ADCY9|115,ADCYAP1|116,ADCYAP1R1|117,ADD1|118,ADD2|119,ADD3|120,ADH1A|124,ADH1B|125,

ABSOLUTE shape: (837726, 20)


,Sample,Chromosome,Start,End,Num_Probes,Length,Modal_HSCN_1,Modal_HSCN_2,Modal_Total_CN,Subclonal_HSCN_a1,Subclonal_HSCN_a2,Cancer_cell_frac_a1,Ccf_ci95_low_a1,Ccf_ci95_high_a1,Cancer_cell_frac_a2,Ccf_ci95_low_a2,Ccf_ci95_high_a2,LOH,Homozygous_deletion,solution
0,TCGA-OR-A5J1-01,1.0,564621.0,8644007.0,2257.0,8079386.0,1.0,1.0,2.0,0.0,0.0,0.05,0.02170,0.07127,0.05,0.02170,0.07127,0.0,0.0,new
1,TCGA-OR-A5J1-01,1.0,8652154.0,9140078.0,121.0,487924.0,1.0,2.0,3.0,0.0,0.0,0.08,0.00356,0.14847,0.95,0.86228,1.00000,0.0,0.0,new
2,TCGA-OR-A5J1-01,1.0,9143332.0,12837840.0,848.0,3694508.0,1.0,1.0,2.0,0.0,0.0,0.06,0.02493,0.08426,0.06,0.02493,0.08426,0.0,0.0,new
3,TCGA-OR-A5J1-01,1.0,12852748.0,16846035.0,1120.0,3993287.0,1.0,1.0,2.0,0.0,0.0,0.03,0.00000,0.04964,0.03,0.00000,0.04964,0.0,0.0,new
4,TCGA-OR-A5J1-01,1.0,17246171.0,25570830.0,2789.0,8324659.0,1.0,1.0,2.0,0.0,0.0,0.04,0.01486,0.06423,0.04,0.01476,0.06412,0.0,0.0,new


CDR shape: (11160, 34)


,Unnamed: 0,bcr_patient_barcode,type,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,initial_pathologic_dx_year,menopause_status,birth_days_to,vital_status,tumor_status,last_contact_days_to,death_days_to,cause_of_death,new_tumor_event_type,new_tumor_event_site,new_tumor_event_site_other,new_tumor_event_dx_days_to,treatment_outcome_first_course,margin_status,residual_tumor,OS,OS.time,DSS,DSS.time,DFI,DFI.time,PFI,PFI.time,Redaction
0,1,TCGA-OR-A5J1,ACC,58.0,MALE,WHITE,Stage II,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2000.0,[Not Available],-21496.0,Dead,WITH TUMOR,NaN,1355.0,[Not Available],Distant Metastasis,Peritoneal Surfaces,NaN,754.0,Complete Remission/Response,NaN,NaN,1.0,1355.0,1.0,1355.0,1.0,754.0,1.0,754.0,NaN
1,2,TCGA-OR-A5J2,ACC,44.0,FEMALE,WHITE,Stage IV,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2004.0,[Not Available],-16090.0,Dead,WITH TUMOR,NaN,1677.0,[Not Available],Distant Metastasis,Soft Tissue,NaN,289.0,Progressive Disease,NaN,NaN,1.0,1677.0,1.0,1677.0,NaN,NaN,1.0,289.0,NaN
2,3,TCGA-OR-A5J3,ACC,23.0,FEMALE,WHITE,Stage III,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2008.0,[Not Available],-8624.0,Alive,WITH TUMOR,2091.0,NaN,[Not Available],Distant Metastasis,Lung,NaN,53.0,Complete Remission/Response,NaN,NaN,0.0,2091.0,0.0,2091.0,1.0,53.0,1.0,53.0,NaN
3,4,TCGA-OR-A5J4,ACC,23.0,FEMALE,WHITE,Stage IV,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2000.0,[Not Available],-8451.0,Dead,WITH TUMOR,NaN,423.0,[Not Available],Locoregional Recurrence,Peritoneal Surfaces,NaN,126.0,Progressive Disease,NaN,NaN,1.0,423.0,1.0,423.0,NaN,NaN,1.0,126.0,NaN
4,5,TCGA-OR-A5J5,ACC,30.0,MALE,WHITE,Stage III,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2000.0,[Not Available],-11171.0,Dead,WITH TUMOR,NaN,365.0,[Not Available],Locoregional Recurrence,"Other, specify",vena cava thrombus,50.0,Progressive Disease,NaN,NaN,1.0,365.0,1.0,365.0,NaN,NaN,1.0,50.0,NaN


## Exploring the preprocessed dataset

### RNA data

Description

#### Basic stats

In [5]:
rna_df.dtypes

gene_id
?|100130426      float64
?|100133144      float64
?|100134869      float64
?|10357          float64
?|10431          float64
                  ...   
ZYG11A|440590    float64
ZYG11B|79699     float64
ZYX|7791         float64
ZZEF1|23140      float64
ZZZ3|26009       float64
Length: 20531, dtype: object

In [6]:
rna_df.nunique()

gene_id
?|100130426        447
?|100133144      10228
?|100134869      10582
?|10357          10939
?|10431          10974
                 ...  
ZYG11A|440590     9669
ZYG11B|79699     10950
ZYX|7791         10992
ZZEF1|23140      10899
ZZZ3|26009       10982
Length: 20531, dtype: int64

In [7]:
# Show % of missing values per column, sorted descending
missing_percent = rna_df.isnull().mean().sort_values(ascending=False) * 100
missing_percent = missing_percent[missing_percent > 0]  # Only show columns with missing data

# Display results
print(f"Number of columns with missing values: {missing_percent.shape[0]}")
display(missing_percent.to_frame(name="Missing (%)"))


Number of columns with missing values: 4196


,Missing (%)
gene_id,
CT47A10|728036,15.818954
SNORD76|692196,15.818954
SNORD92|692209,15.818954
MAGEB3|4114,15.818954
SNORD91B|692208,15.818954
...,...
LY6G6E|79136,0.298130
LOC100124692|100124692,0.298130
TMEM72|643236,0.298130


No gene has more than 16% missing values.

In [8]:
# rna_df = du.data_processing.standardize_missing_values_df(rna_df)
# rna_df.head()

In [9]:
# du.search_explore.dataframe_missing_values(rna_df)

The missing values standardization process with take around 30 hours to complete (on Paperspace's C7 machine)! Still, it seems like this table has the right missing values representation, so we don't need to run these last two cells.

In [10]:
rna_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
gene_id,,,,,,,,
?|100130426,9351.0,0.028386,0.239364,-0.090423,0.00000,0.000000,0.000000,17.995000
?|100133144,11069.0,11.251089,12.199371,-0.525753,3.44920,7.809006,14.976397,187.474945
?|100134869,11069.0,11.670693,11.671968,-0.516280,4.59750,8.640000,14.987820,402.218988
?|10357,11069.0,160.950078,126.505719,0.000000,85.28260,123.578000,191.223000,1995.021959
?|10431,11069.0,986.122273,416.277222,232.315000,705.77300,905.364000,1172.160000,6818.118303
...,...,...,...,...,...,...,...,...
ZYG11A|440590,11069.0,55.909279,173.232482,-0.402069,1.28640,11.078700,58.539600,9050.870000
ZYG11B|79699,11069.0,924.782456,480.222826,46.019981,634.85406,834.735000,1114.529951,21058.900000
ZYX|7791,11069.0,4669.085953,3148.075907,139.635000,2685.23000,3983.570000,5729.070000,49294.800000


The data is not (well) normalized yet. All columns should have 0 mean and 1 standard deviation.

### ABSOLUTE-annotated seg data

This dataframe contains copy-number and copy-ratio related data.

Copy number alterations/aberrations (CNAs) are changes in copy number that have arisen in somatic tissue (for example, just in a tumor), copy number variations (CNVs) originated from changes in copy number in germline cells (and are thus in all cells of the organism).

#### Basic stats

In [11]:
abs_df.dtypes

Sample                  object
Chromosome             float64
Start                  float64
End                    float64
Num_Probes             float64
Length                 float64
Modal_HSCN_1           float64
Modal_HSCN_2           float64
Modal_Total_CN         float64
Subclonal_HSCN_a1      float64
Subclonal_HSCN_a2      float64
Cancer_cell_frac_a1    float64
Ccf_ci95_low_a1        float64
Ccf_ci95_high_a1       float64
Cancer_cell_frac_a2    float64
Ccf_ci95_low_a2        float64
Ccf_ci95_high_a2       float64
LOH                    float64
Homozygous_deletion    float64
solution                object
dtype: object

In [12]:
abs_df.nunique()

Sample                   4944
Chromosome                 22
Start                  236259
End                    236237
Num_Probes              37353
Length                 533502
Modal_HSCN_1                8
Modal_HSCN_2                8
Modal_Total_CN              8
Subclonal_HSCN_a1           2
Subclonal_HSCN_a2           2
Cancer_cell_frac_a1       101
Ccf_ci95_low_a1         82713
Ccf_ci95_high_a1        85868
Cancer_cell_frac_a2       101
Ccf_ci95_low_a2         84845
Ccf_ci95_high_a2        91175
LOH                         2
Homozygous_deletion         2
solution                    1
dtype: int64

In [13]:
abs_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Chromosome,837726.0,9.263117e+00,6.019021e+00,1.00000,4.000000e+00,8.000000e+00,1.400000e+01,2.200000e+01
Start,837726.0,6.450491e+07,5.681006e+07,6888.00000,1.921226e+07,4.975886e+07,9.964193e+07,2.491582e+08
End,837725.0,8.070407e+07,5.895394e+07,52368.00000,3.329391e+07,6.766270e+07,1.196756e+08,2.491987e+08
Num_Probes,837725.0,4.986391e+03,9.232785e+03,10.00000,2.140000e+02,1.298000e+03,5.427000e+03,7.250600e+04
Length,837725.0,1.619926e+07,3.036585e+07,1382.00000,7.697590e+05,4.424068e+06,1.705390e+07,2.445935e+08
Modal_HSCN_1,837725.0,9.391519e-01,6.925746e-01,0.00000,1.000000e+00,1.000000e+00,1.000000e+00,7.000000e+00
Modal_HSCN_2,837725.0,1.873334e+00,1.306222e+00,0.00000,1.000000e+00,1.000000e+00,2.000000e+00,7.000000e+00
Modal_Total_CN,837725.0,2.761035e+00,1.480728e+00,0.00000,2.000000e+00,2.000000e+00,3.000000e+00,7.000000e+00
Subclonal_HSCN_a1,837725.0,1.488758e-01,3.559662e-01,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
Subclonal_HSCN_a2,837725.0,1.985353e-01,3.988975e-01,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00


The data is not (well) normalized yet. All columns should have 0 mean and 1 standard deviation.

In [14]:
abs_df.solution.value_counts()

solution
new    837725
Name: count, dtype: int64

Columns `Start`, `End` and `Num_Probes` don't seem to be relevant in this stationary (not temporal) scenario, without the need for experiment specific information.

### Clinical outcome (TCGA-CDR) data

Description

**Comments:**
* Features such as `age_at_initial_pathologic_diagnosis`, `gender`, `race`, `ajcc_pathologic_tumor_stage`, `vital_status` and `tumor_status` might be very insteresting to include. Others such as overall survival (OS), progression-free interval (PFI), disease-free interval (DFI), and disease-specific survival (DSS) might not be relevant for this use case.
* Missing values appear to be mostly represented as "[Not Applicable]", "[Not Available]", "[Not Evaluated]", "[Unknown]" or "[Discrepancy]".
* Features related to outcomes, such as `treatment_outcome_first_course` and `death_days_to`, should be ignored, as we're classifying tumor type, regardless of the outcome.

#### Basic stats

In [15]:
cdr_df.dtypes

Unnamed: 0                               int64
bcr_patient_barcode                     object
type                                    object
age_at_initial_pathologic_diagnosis    float64
gender                                  object
race                                    object
ajcc_pathologic_tumor_stage             object
clinical_stage                          object
histological_type                       object
histological_grade                      object
initial_pathologic_dx_year             float64
menopause_status                        object
birth_days_to                          float64
vital_status                            object
tumor_status                            object
last_contact_days_to                   float64
death_days_to                          float64
cause_of_death                          object
new_tumor_event_type                    object
new_tumor_event_site                    object
new_tumor_event_site_other              object
new_tumor_eve

In [16]:
cdr_df.nunique()

Unnamed: 0                             11160
bcr_patient_barcode                    11160
type                                      33
age_at_initial_pathologic_diagnosis       78
gender                                     2
race                                       8
ajcc_pathologic_tumor_stage               23
clinical_stage                            34
histological_type                        144
histological_grade                        11
initial_pathologic_dx_year                33
menopause_status                           7
birth_days_to                           8280
vital_status                               3
tumor_status                               4
last_contact_days_to                    2755
death_days_to                           1680
cause_of_death                            13
new_tumor_event_type                      35
new_tumor_event_site                      61
new_tumor_event_site_other               367
new_tumor_event_dx_days_to              1304
treatment_

Most of the features with significant quantities of missing values (>40%) are not going to be used. But it's important to remember that categorical features like `clinical_stage` use different representations for missing values.

In [17]:
cdr_df.vital_status.value_counts()

vital_status
Alive            7528
Dead             3627
[Discrepancy]       1
Name: count, dtype: int64

Should probably assume the "[Discrepancy]" option as a missing value also.

In [18]:
cdr_df.ajcc_pathologic_tumor_stage.value_counts()

ajcc_pathologic_tumor_stage
[Not Available]     2280
[Not Applicable]    1864
Stage I             1286
Stage IIA            804
Stage III            709
Stage IIB            702
Stage II             642
Stage IIIA           507
Stage IV             479
Stage IB             401
Stage IA             383
Stage IVA            336
Stage IIIB           286
Stage IIIC           248
Stage IIC             67
[Discrepancy]         53
IS                    46
Stage IVB             21
I/II NOS              14
Stage X               13
Stage IVC              7
Stage 0                7
[Unknown]              5
Name: count, dtype: int64

In [19]:
cdr_df.clinical_stage.value_counts()

clinical_stage
[Not Applicable]    7003
[Not Available]     1521
Stage IIIC           458
Stage IVA            283
Stage IA             214
Stage IB             205
Stage I              190
Stage II             183
Stage III            156
Stage IV             128
Stage IIB            103
Stage IIIB            94
Stage IIIA            91
Stage IB1             78
Stage IIA             57
Stage IVB             56
Stage IS              42
IIa                   41
Stage IB2             39
Stage IC              38
I                     38
Stage IIIC2           28
Stage IIIC1           26
Stage IIC             22
IIb                   20
III                   15
Stage IVC              7
Stage IIA2             7
IVb                    6
Stage IIA1             5
[Discrepancy]          2
IVa                    2
Stage IA1              1
Stage IA2              1
Name: count, dtype: int64

Considerable percentage of missing values on `ajcc_pathologic_tumor_stage` (\~37%) and `clinical_stage` (\~76%).

Considering the real percentages of missing values, which are higher than what we got before standardizing the missing values representation, the main features to use from this table should be `gender`, `vital_status`, `age_at_initial_pathologic_diagnosis`, `tumor_status`, `race` and `ajcc_pathologic_tumor_stage`.

In [20]:
cdr_df.ajcc_pathologic_tumor_stage.value_counts()

ajcc_pathologic_tumor_stage
[Not Available]     2280
[Not Applicable]    1864
Stage I             1286
Stage IIA            804
Stage III            709
Stage IIB            702
Stage II             642
Stage IIIA           507
Stage IV             479
Stage IB             401
Stage IA             383
Stage IVA            336
Stage IIIB           286
Stage IIIC           248
Stage IIC             67
[Discrepancy]         53
IS                    46
Stage IVB             21
I/II NOS              14
Stage X               13
Stage IVC              7
Stage 0                7
[Unknown]              5
Name: count, dtype: int64

In [21]:
cdr_df.clinical_stage.value_counts()

clinical_stage
[Not Applicable]    7003
[Not Available]     1521
Stage IIIC           458
Stage IVA            283
Stage IA             214
Stage IB             205
Stage I              190
Stage II             183
Stage III            156
Stage IV             128
Stage IIB            103
Stage IIIB            94
Stage IIIA            91
Stage IB1             78
Stage IIA             57
Stage IVB             56
Stage IS              42
IIa                   41
Stage IB2             39
Stage IC              38
I                     38
Stage IIIC2           28
Stage IIIC1           26
Stage IIC             22
IIb                   20
III                   15
Stage IVC              7
Stage IIA2             7
IVb                    6
Stage IIA1             5
[Discrepancy]          2
IVa                    2
Stage IA1              1
Stage IA2              1
Name: count, dtype: int64

In [22]:
cdr_df.race.value_counts()

race
WHITE                                        8186
[Not Available]                              1038
BLACK OR AFRICAN AMERICAN                     934
ASIAN                                         675
[Not Evaluated]                               155
[Unknown]                                     132
AMERICAN INDIAN OR ALASKA NATIVE               27
NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER      13
Name: count, dtype: int64

In [23]:
cdr_df.histological_grade.value_counts()

histological_grade
[Not Available]    6515
G3                 2035
G2                 1652
High Grade          399
G1                  319
GX                  109
G4                  104
Low Grade            21
[Unknown]             3
GB                    2
[Discrepancy]         1
Name: count, dtype: int64

In [24]:
cdr_df.vital_status.value_counts()

vital_status
Alive            7528
Dead             3627
[Discrepancy]       1
Name: count, dtype: int64

In [25]:
cdr_df.margin_status.value_counts()

margin_status
Negative           1105
Positive            113
[Not Available]     102
Close                23
[Unknown]            15
Name: count, dtype: int64

Not sure what `margin_status` represents.

In [26]:
cdr_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,11160.0,5580.500000,3221.758836,1.0,2790.75,5580.5,8370.25,11160.0
age_at_initial_pathologic_diagnosis,11109.0,59.099019,14.415440,10.0,50.00,60.0,70.00,90.0
initial_pathologic_dx_year,11030.0,2008.033545,4.321909,1978.0,2006.00,2009.0,2011.00,2013.0
birth_days_to,11041.0,-21762.930260,5266.388866,-32918.0,-25609.00,-22222.0,-18516.00,-3982.0
last_contact_days_to,8254.0,1088.739763,1049.365330,-64.0,414.00,766.0,1445.50,11252.0
death_days_to,3597.0,853.025021,949.406905,0.0,276.00,548.0,1096.00,10870.0
new_tumor_event_dx_days_to,3346.0,594.534071,751.776619,-359.0,190.25,364.0,693.75,10334.0
OS,11151.0,0.324814,0.468326,0.0,0.00,0.0,1.00,1.0
OS.time,11094.0,1038.605553,1038.663449,0.0,381.00,714.0,1369.00,11252.0
DSS,10648.0,0.234692,0.423826,0.0,0.00,0.0,0.00,1.0


#### Tumor representation

In [27]:
cdr_df.type.value_counts().to_frame()

,count
type,
BRCA,1097
GBM,596
OV,587
UCEC,548
KIRC,537
HNSC,528
LUAD,522
LGG,515
THCA,507
